In [335]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [336]:
if "train_x" not in globals():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))



In [337]:
lib.utils.set_seed()
# network = lib.nets.PYRAMID()
#network = lib.nets.create_livenet_odd_2()
# train_x, train_y = lib.datasets.get_odd()
# network = lib.livenet.LiveNet(train_x.shape[1], None, torch.max(train_y) + 1)
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1, l1=0.01)
criterion = lib.nets.criterion_n

def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net, decay=0.0)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        # optimizer = torch.optim.Adam(net.parameters(), betas=(0.0, 0.95))
        optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.Adam, net.parameters(), betas=(0.0, 0.95))
    return optimizer

optimizer = create_optimizer(network)



Torch


In [339]:
lib.utils.set_seed()
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1, l1=0.01)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = create_optimizer(network)
optimizer.learning_rate = 0.0001
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=60, adaptive_lr=True)


Torch


In [340]:

trainer.step(601)


Iˈ0.000 1.391 = 1.105+0.286 lr=0.00010 gratio=0.000 lib/trainer.py:82
Iˈ0.100 1.156 = 0.883+0.273 lr=0.00033 gratio=1.000 lib/trainer.py:82
Iˈ0.193 0.843 = 0.608+0.235 lr=0.00061 gratio=0.917 lib/trainer.py:82
Iˈ0.291 0.709 = 0.515+0.193 lr=0.00063 gratio=0.833 lib/trainer.py:82
Iˈ0.385 0.671 = 0.509+0.163 lr=0.00103 gratio=0.900 lib/trainer.py:82
Iˈ0.480 0.619 = 0.482+0.137 lr=0.00068 gratio=0.767 lib/trainer.py:82
Iˈ0.574 0.608 = 0.485+0.122 lr=0.00088 gratio=0.867 lib/trainer.py:82
Iˈ0.658 0.595 = 0.480+0.115 lr=0.00073 gratio=0.800 lib/trainer.py:82
Iˈ0.734 0.590 = 0.479+0.112 lr=0.00085 gratio=0.850 lib/trainer.py:82
Iˈ0.820 0.595 = 0.483+0.112 lr=0.00078 gratio=0.817 lib/trainer.py:82
Iˈ0.898 0.580 = 0.471+0.109 lr=0.00065 gratio=0.800 lib/trainer.py:82


In [73]:
params_history = []
optimizer.param_groups[0]["lr"]



In [71]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.1385

In [72]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["linear1.weight"][0][1].item()
    return val0
    return param["grads"]["linear1.weight"][0][0].item() / 1000
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker0)
# plt.plot(values)
values = get_param_values(trainer.history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()




Using matplotlib backend: TkAgg


In [39]:
def form_balanced(x, y, n):
    assert n % 2 == 0
    n //= 2
    y = y.numpy()
    x = x.numpy()
    inds = np.argsort(y.squeeze(1), axis=0)
    y = y[inds]
    x = x[inds]
    i = 0
    batches_x = []
    batches_y = []
    while True:
        a = y[i:i + n]
        b = y[len(y) - i - n: len(y) - i]
        if not (a==0).all() or not (b==1).all():
            break
        batches_x += [x[i:i+n], x[len(y) - i - n: len(y) - i]]
        batches_y += [a, b]
        i += n

    x = np.vstack(batches_x)
    y = np.vstack(batches_y)
    return torch.tensor(x), torch.tensor(y)

train_x, train_y = form_balanced(train_x, train_y, 1000)
